In [1]:
'''
tensorflow = 2.8.0
keras = 2.6.0
transformers = 4.11.3
'''

'\ntensorflow = 2.8.0\nkeras = 2.6.0\ntransformers = 4.11.3\n'

In [2]:
# !pip install -U transformers

In [3]:
!nvidia-smi

Tue Nov 29 04:50:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    39W / 300W |      0MiB / 16384MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
import sys
sys.path.append(os.getenv('HOME')+'/aiffelthon')
print(sys.path)

['/home/wlsaud5474/aiffelthon/notebook', '/home/wlsaud5474/anaconda3/envs/jinmyeong/lib/python37.zip', '/home/wlsaud5474/anaconda3/envs/jinmyeong/lib/python3.7', '/home/wlsaud5474/anaconda3/envs/jinmyeong/lib/python3.7/lib-dynload', '', '/home/wlsaud5474/anaconda3/envs/jinmyeong/lib/python3.7/site-packages', '/home/wlsaud5474/anaconda3/envs/jinmyeong/lib/python3.7/site-packages/IPython/extensions', '/home/wlsaud5474/.ipython', '/home/wlsaud5474/aiffelthon']


In [5]:
# from bert import modeling_bert

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tqdm
from transformers import Trainer, TrainingArguments
import re
import pyarrow as pa
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.model_selection import train_test_split
from datasets import load_metric

In [7]:
import os
data_dir = os.getenv('HOME')+'/aiffelthon/AIFFELTHON/TUNiB/experiment_data'
data = pd.read_csv(f"{data_dir}/train_bt_en_fr_ch_8000.csv")
data

,class,non_label_sentence
0,직장 내 괴롭힘 대화,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...
1,기타 괴롭힘 대화,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...
2,갈취 대화,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...
3,갈취 대화,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...
4,협박 대화,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...
...,...,...
26970,일반 대화,혹시 자기소개서 써봤어\n한번도 안 써봤어\n자기소개서 쓰는 것 정말 힘들어\n너는...
26971,일반 대화,나 이번에 화상으로 수업 듣다가 졸았잖아 키키\n키키 인강 안 듣다가 왠일로 들었어...
26972,일반 대화,비행기 타 본 적 있어\n나 고등학교 졸업여행 때 한 번\n오 졸업여행도 갔었나\n...
26973,일반 대화,너는 학생 때 어떤 쌤이 제일 좋았어\n나는 별로 좋았던 쌤이 없었어\n중학생 ...


In [8]:
# data.drop(['label_sentence', 'binary_class'], axis=1, inplace=True)

In [9]:

labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화':4}
num_labels = len(labels)
data['label'] = data['class'].apply(lambda x : labels[x])
data.head(10)

,class,non_label_sentence,label
0,직장 내 괴롭힘 대화,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,2
1,기타 괴롭힘 대화,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,3
2,갈취 대화,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,1
3,갈취 대화,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,1
4,협박 대화,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,0
5,갈취 대화,야 돼지 오늘도 돈 내놔\n오늘은 진짜 없어\n아새끼 오늘 진짜 돈 안 들고 왔네\...,1
6,기타 괴롭힘 대화,오 신발 샀냐\n응 전에 신던 게 다 헤져서 부모님이 사주셨어\n너네 집 꽤 사...,3
7,기타 괴롭힘 대화,영하 학생 이리 나와요\n네 선생님\n선생님이 왜 부른 줄 알아\n잘 모르겠습니다 ...,3
8,직장 내 괴롭힘 대화,씨 오늘 뭐 해요\n아 저 약속이 있어서요\n누구랑요 남자친구\n아 네\n에이 ...,2
9,갈취 대화,야 죽기 싫으면 돈 내놔\n죄송합니다 여기\n뭔 이거 밖 없어 뒤져서 나오면 죽...,1


In [10]:
from transformers import AutoTokenizer, AutoModel

from transformers import ElectraForSequenceClassification, RobertaForSequenceClassification
from bert.modeling_bert import BertForSequenceClassification 

HUGGINGFACE_MODEL_PATH = "klue/bert-base"
# "klue/roberta-large"
# "beomi/kcbert-large"
# "beomi/KcELECTRA-base-v2022"
# "klue/bert-base"

tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
#model = AutoModel.from_pretrained("beomi/KcELECTRA-base")
model = BertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=num_labels)


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [11]:
data = Dataset(pa.Table.from_pandas(data))

In [12]:
def transform(data):
  return tokenizer(
      data['non_label_sentence'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_data = data.map(transform, batched=True)

  0%|          | 0/27 [00:00<?, ?ba/s]

In [13]:
train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()

In [14]:
train_dataset

Dataset({
    features: ['class', 'non_label_sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 24277
})

In [15]:
metric = load_metric("f1")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

output_dir = os.getenv('HOME')+'/aiffelthon/AIFFELTHON/Fine_tuning/KcBERT-large'


training_arguments = TrainingArguments(
    output_dir, 
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, 
    per_device_train_batch_size = 8, 
    per_device_eval_batch_size = 8, 
    num_train_epochs = 2,
    weight_decay = 0.01
)

/home/wlsaud5474/anaconda3/envs/jinmyeong/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [16]:
trainer = Trainer(
    model=model,                           
    args=training_arguments,                  
    train_dataset=train_dataset,    
    eval_dataset=validation_dataset,      
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: class, non_label_sentence. If class, non_label_sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/wlsaud5474/anaconda3/envs/jinmyeong/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 24277
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6070
  Number of trainable parameters = 110621189


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## Test

In [ ]:
test_file_path = os.getenv('HOME')+'/aiffelthon/AIFFELTHON/TUNiB/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)

test_data = test_dataset.transpose()
test_data

In [ ]:
import re


def remove_punctuation(x):
  x = x.strip()
  x = re.sub("[^가-힣0-9]+", " ", x)
  x = re.sub("[ ]+", " ", x)
  x = x.strip()
  return x

test_data["text"] = test_data['text'].apply(lambda x : remove_punctuation(x))
test_data.head()

In [ ]:
test = Dataset(pa.Table.from_pandas(test_data))

In [ ]:
def transform_test(data):
  return tokenizer(
      data['text'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_test = test.map(transform_test, batched=True)
encoded_test

In [ ]:
pred = trainer.predict(encoded_test)

In [ ]:
pred_label = []

for i in range(len(pred[0])):
    pred_label.append((str)(np.argmax(pred[0][i])).zfill(2))
    
pred_label[:10]

In [ ]:
submission = test_data.assign(CLASS=pred_label)
submission = submission.rename(columns={'CLASS':'class'})

submission.drop(['text'], axis=1, inplace=True)
submission = submission.transpose()
submission

In [ ]:
# import json
# sub_dir = os.getenv('HOME')+'/aiffelthon/AIFFELTHON/TUNiB/submission'
# submission_file_path = f'{sub_dir}/submission_klue-bert-base-en_fr_ch_8000.json'
# result = submission.to_json(submission_file_path)

# with open(submission_file_path) as f:
#     parsed = json.load(f)

# with open(submission_file_path, 'w') as f:
#     json.dump(parsed, f, indent=4)